In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 22 16:04:07 2020

@author: dschroeder

"""

# Imports
import numpy as np
import random as rd
import math
import IOHexperimenter as IOH
from IOHexperimenter import IOH_function

#setting the random seed
rd.seed()

class simple_EA:
    
    def __init__(self, budget):
        self.budget = budget
        self.f_opt = np.Inf
        self.x_opt = None
        self.d = 1
                
        # Parameter setting
        self.mu = 15 #number of parents
        self.lamda = 40 #number of offspring
        self.pc = 0.8
        self.tau = 1 / math.sqrt(self.d)
        self.sigma = 10/6
        
    def initialize_individual(self):
        individual = np.empty((self.d,1))
        for i in range (0,self.d):
            individual[i]= rd.uniform(-5, 5)
        return individual

    def initialize_population(self):
        population = np.zeros((self.mu,self.d))
        for k in range(0,self.mu):
            population[k] = np.transpose(self.initialize_individual())
        return population

    def recombine(self,pop):
        offspring = np.zeros((self.d))
        p1 = pop[rd.randrange(0,len(pop))]
        p2 = pop[rd.randrange(0,len(pop))]
        if rd.uniform(0,1) < self.pc:
            for k in range (0,self.d):
                random_number = rd.uniform(0,1)
                if random_number < 0.5:
                    offspring[k] = p1[k]
                else:
                    offspring[k] = p2[k]
        else:
            offspring = p1
        return offspring

    def update_sigma(self):
        self.sigma = self.sigma * math.exp(self.tau * np.random.normal(0,1))
        return self.sigma

    def singlestep_mutate (self,ind):
        mutant = np.zeros(len(ind))
        for j in range (0, len(ind)):
             result = ind[j] + self.sigma * np.random.normal(0,1)
             # check boundaries first
             if result < -5:
                  mutant[j] = -5
             elif result > 5:
                 mutant[j] = 5
             # if result is within boundaries, change j-th mutant characteristic to result
             else:
                 mutant[j] = result

        return mutant
  
    def __call__(self, func):
        
        evalcount = 0
        self.d = func.number_of_variables
        
        #Initialization
        self.xp = self.initialize_population()
        self.f = np.zeros((self.mu,1))
        self.xo = np.zeros((self.lamda,self.d))
        self.fo = np.zeros((self.lamda,1))
        
        for i in range (0,self.mu):
            self.f[i] = func(self.xp[i])
            evalcount = evalcount +1
        
        # Evaluation loop
        while evalcount < self.budget:
            self.sigma = self.update_sigma()
            for i in range(0,self.lamda):
                offspring = self.recombine(self.xp)
                self.xo[i] = self.singlestep_mutate(offspring)
                self.fo[i] = func(self.xo[i])
                evalcount = evalcount +1
                
            # Selection
            m = np.concatenate((self.xo, self.fo), axis=1) # Create joint matrix of individuals and their fitness values
            sorted_m = m[np.argsort(m[:, self.d])] # sort based on fitness (lowest values highest ranking)
            xpnew = sorted_m[0:self.mu,0:self.d] # copy best individuals and remove fitness values

            self.xp = xpnew # set new population for next generation
            self.f = sorted_m[0:self.mu,self.d] # copy fitness values of new generation

            self.x_opt = xpnew[0]
            self.f_opt = self.f[0]

        return self.x_opt, self.f_opt

In [26]:
list = []
for i in range (1,25):
    list.append(i)

In [32]:
from IOHexperimenter import IOHexperimenter
exp = IOHexperimenter()

exp.initialize_BBOB(list,[1],[2,5,20], 1)
exp.set_logger_location("experiment3", "run")

In [33]:
exp([simple_EA(1000)])

Running single-threaded
1 1015 0.003230987358201044
1 1015 0.8916228920652869
1 1015 27.149108645929367
2 1015 2360.235694563326
2 1015 493640.06810779305
2 1015 3221632.4269640637
3 1015 24.03371810497678
3 1015 85.09594755852024
3 1015 1134.1919054602354
4 1015 36.43764124822604
4 1015 103.99887799928737
4 1015 1219.844797847963
5 1015 0.0
5 1015 0.0
5 1015 69.21316810795969
6 1015 1645.06095933472
6 1015 48.632474316512955
6 1015 2403527.18006115
7 1015 94.62349805255676
7 1015 122.04465316036989
7 1015 1082.0687962543057
8 1015 30.31197930875793
8 1015 387.4287072262493
8 1015 247487.42966616334
9 1015 48.163288226713945
9 1015 6812.768812894533
9 1015 155227.32979205428
10 1015 30229.898242621508
10 1015 192546.15015978873
10 1015 2571622.518682225
11 1015 875.2792652326605
11 1015 96219.0493553685
11 1015 650.955960906523
12 1015 959468.247465836
12 1015 26626788.47193153
12 1015 398944371.7431184
13 1015 119.68200248448493
13 1015 1042.9361883422441
13 1015 2490.7898329492336
14

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [29]:
?exp.initialize_BBOB

Signature: exp.initialize_BBOB(fids, iids, dims, repetitions, precision=1e-08)
Docstring:
Initialize to a set of functions for the BBOB suite

Parameters
----------
fids:
    List of the function numbers within the selected suite. Alternatively, the names of the functions as used in IOHexperimenter.
dims:
    List of the dimensions (number of variables) of the problem
iids:
    List of the instance IDs of the problem
repetitions: 
    Number of repetitions on each function instance
target_precision:
    Optional, how close to the optimum the problem is considered 'solved'
File:      /usr/local/Cellar/python@3.8/3.8.5/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IOHexperimenter/src/IOH_benchmarking.py
Type:      method


[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]